# MD Analysis Notebook

This notebook provides functions for analyzing molecular dynamics trajectories:
- RMSD (Root Mean Square Deviation)
- RMSF (Root Mean Square Fluctuation)
- RGYR (Radius of Gyration)

Provide the paths to your PSF and trajectory files to analyze your molecular dynamics simulations.

In [ ]:
# Import required libraries
import MDAnalysis as mda
from MDAnalysis.analysis.rms import RMSD, RMSF
import matplotlib.pyplot as plt
import pandas as pd
import os
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# Create file input widgets
psf_file_input = widgets.Text(
    value='',
    placeholder='Enter path to PSF file',
    description='PSF File:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

traj_file_input = widgets.Text(
    value='',
    placeholder='Enter path to trajectory file (DCD or XTC)',
    description='Trajectory File:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

display(psf_file_input)
display(traj_file_input)

In [ ]:
def check_input_files():
    """Check if input files exist and return True if they do, False otherwise"""
    psf_file = psf_file_input.value
    traj_file = traj_file_input.value
    
    valid = True
    if not psf_file:
        print("Error: Please enter a path to a PSF file.")
        valid = False
    elif not os.path.exists(psf_file):
        print(f"Error: PSF file {psf_file} not found")
        valid = False
    
    if not traj_file:
        print("Error: Please enter a path to a trajectory file.")
        valid = False
    elif not os.path.exists(traj_file):
        print(f"Error: Trajectory file {traj_file} not found")
        valid = False
    
    if valid:
        # Create output directory
        traj_dir = os.path.dirname(traj_file)
        output_dir = os.path.join(traj_dir, "output")
        os.makedirs(output_dir, exist_ok=True)
        
        # Get base name of trajectory file for output naming
        traj_base = os.path.splitext(os.path.basename(traj_file))[0]
        
        return True, psf_file, traj_file, output_dir, traj_base
    
    return False, None, None, None, None

## RMSD Analysis (Root Mean Square Deviation)

RMSD measures the average distance between atoms of superimposed structures.

In [ ]:
def run_rmsd_analysis():
    valid, psf_file, traj_file, output_dir, traj_base = check_input_files()
    if not valid:
        return
    
    try:
        # Load data
        u = mda.Universe(psf_file, traj_file)

        # Run RMSD analysis
        rmsd = RMSD(u, select="name CA")  # Select only CA atoms
        rmsd.run()

        # Create DataFrame
        df = pd.DataFrame({
            "Time (ps)": rmsd.results.rmsd[:, 1],
            "RMSD (Å)": rmsd.results.rmsd[:, 2]
        })

        # Save data to CSV in output directory
        csv_path = os.path.join(output_dir, f"{traj_base}_rmsd.csv")
        df.to_csv(csv_path, index=False)
        print(f"CSV data saved to: {csv_path}")

        # Create plot
        plt.figure(figsize=(10, 6))
        plt.plot(df["Time (ps)"], df["RMSD (Å)"])
        plt.xlabel("Time (ps)")
        plt.ylabel("RMSD (Å)")
        plt.title(f"Backbone RMSD - {traj_base}")

        # Save plot to output directory
        plot_path = os.path.join(output_dir, f"{traj_base}_rmsd_plot.png")
        plt.savefig(plot_path, dpi=300)
        print(f"Plot saved to: {plot_path}")

        # Show the plot
        plt.show()
        
        return df
    except Exception as e:
        print(f"Error running RMSD analysis: {e}")
        return None

In [ ]:
# Run RMSD Analysis Button
rmsd_button = widgets.Button(
    description='Run RMSD Analysis',
    button_style='success', 
    tooltip='Click to run RMSD analysis'
)

def on_rmsd_button_clicked(b):
    run_rmsd_analysis()
    
rmsd_button.on_click(on_rmsd_button_clicked)
display(rmsd_button)

## RMSF Analysis (Root Mean Square Fluctuation)

RMSF measures the deviation of a residue or atom from its average position.

In [ ]:
def run_rmsf_analysis():
    valid, psf_file, traj_file, output_dir, traj_base = check_input_files()
    if not valid:
        return
    
    try:
        # Load data
        u = mda.Universe(psf_file, traj_file)
        ca_atoms = u.select_atoms("name CA")  # Select only CA atoms

        # Run RMSF analysis
        rmsf = RMSF(ca_atoms)
        rmsf.run()

        # Create DataFrame
        df = pd.DataFrame({
            "Residue": ca_atoms.resids,
            "RMSF (Å)": rmsf.results.rmsf
        })

        # Save data to CSV in output directory
        csv_path = os.path.join(output_dir, f"{traj_base}_rmsf.csv")
        df.to_csv(csv_path, index=False)
        print(f"CSV data saved to: {csv_path}")

        # Create plot
        plt.figure(figsize=(10, 6))
        plt.plot(df["Residue"], df["RMSF (Å)"])
        plt.xlabel("Residue")
        plt.ylabel("RMSF (Å)")
        plt.title(f"Per-residue RMSF - {traj_base}")

        # Save plot to output directory
        plot_path = os.path.join(output_dir, f"{traj_base}_rmsf_plot.png")
        plt.savefig(plot_path, dpi=300)
        print(f"Plot saved to: {plot_path}")

        # Show the plot
        plt.show()
        
        return df
    except Exception as e:
        print(f"Error running RMSF analysis: {e}")
        return None

In [ ]:
# Run RMSF Analysis Button
rmsf_button = widgets.Button(
    description='Run RMSF Analysis',
    button_style='success', 
    tooltip='Click to run RMSF analysis'
)

def on_rmsf_button_clicked(b):
    run_rmsf_analysis()
    
rmsf_button.on_click(on_rmsf_button_clicked)
display(rmsf_button)

## RGYR Analysis (Radius of Gyration)

RGYR measures the compactness of a protein structure.

In [ ]:
def run_rgyr_analysis():
    valid, psf_file, traj_file, output_dir, traj_base = check_input_files()
    if not valid:
        return
    
    try:
        # Load data
        u = mda.Universe(psf_file, traj_file)
        protein = u.select_atoms("protein")

        # Calculate RGYR for each frame
        rgyr = []
        times = []

        for ts in u.trajectory:
            rgyr.append(protein.radius_of_gyration())
            times.append(ts.time)

        # Create DataFrame
        df = pd.DataFrame({
            "Time (ps)": times,
            "Radius of Gyration (Å)": rgyr
        })

        # Save data to CSV in output directory
        csv_path = os.path.join(output_dir, f"{traj_base}_rgyr.csv")
        df.to_csv(csv_path, index=False)
        print(f"CSV data saved to: {csv_path}")

        # Create plot
        plt.figure(figsize=(10, 6))
        plt.plot(times, rgyr)
        plt.xlabel("Time (ps)")
        plt.ylabel("Radius of Gyration (Å)")
        plt.title(f"Protein Compactness - {traj_base}")

        # Save plot to output directory
        plot_path = os.path.join(output_dir, f"{traj_base}_rgyr_plot.png")
        plt.savefig(plot_path, dpi=300)
        print(f"Plot saved to: {plot_path}")

        # Show the plot
        plt.show()
        
        return df
    except Exception as e:
        print(f"Error running RGYR analysis: {e}")
        return None

In [ ]:
# Run RGYR Analysis Button
rgyr_button = widgets.Button(
    description='Run RGYR Analysis',
    button_style='success', 
    tooltip='Click to run RGYR analysis'
)

def on_rgyr_button_clicked(b):
    run_rgyr_analysis()
    
rgyr_button.on_click(on_rgyr_button_clicked)
display(rgyr_button)

## Run All Analyses

In [ ]:
run_all_button = widgets.Button(
    description='Run All Analyses',
    button_style='danger', 
    tooltip='Click to run all analyses'
)

def on_run_all_button_clicked(b):
    print("Running RMSD analysis...")
    run_rmsd_analysis()
    print("\nRunning RMSF analysis...")
    run_rmsf_analysis()
    print("\nRunning RGYR analysis...")
    run_rgyr_analysis()
    print("\nAll analyses complete!")
    
run_all_button.on_click(on_run_all_button_clicked)
display(run_all_button)

## Custom Selection Analysis

You can customize atom selections used in the analyses. See the [MDAnalysis selection syntax](https://userguide.mdanalysis.org/stable/selections.html) for more information.

In [ ]:
# Create selection widgets
rmsd_selection = widgets.Text(
    value='name CA',
    placeholder='Enter MDAnalysis selection string for RMSD',
    description='RMSD Selection:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

rmsf_selection = widgets.Text(
    value='name CA',
    placeholder='Enter MDAnalysis selection string for RMSF',
    description='RMSF Selection:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

rgyr_selection = widgets.Text(
    value='protein',
    placeholder='Enter MDAnalysis selection string for RGYR',
    description='RGYR Selection:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

display(widgets.HTML(value="<h3>Custom Selections</h3>"))
display(rmsd_selection)
display(rmsf_selection)
display(rgyr_selection)

In [ ]:
def run_custom_rmsd_analysis():
    valid, psf_file, traj_file, output_dir, traj_base = check_input_files()
    if not valid:
        return
    
    selection = rmsd_selection.value
    if not selection.strip():
        print("Error: Please enter a valid atom selection")
        return
    
    try:
        # Load data
        u = mda.Universe(psf_file, traj_file)

        # Run RMSD analysis
        rmsd = RMSD(u, select=selection)
        rmsd.run()

        # Create DataFrame
        df = pd.DataFrame({
            "Time (ps)": rmsd.results.rmsd[:, 1],
            "RMSD (Å)": rmsd.results.rmsd[:, 2]
        })

        # Save data to CSV in output directory
        selection_name = selection.replace(" ", "_").replace("*", "all")
        csv_path = os.path.join(output_dir, f"{traj_base}_rmsd_{selection_name}.csv")
        df.to_csv(csv_path, index=False)
        print(f"CSV data saved to: {csv_path}")

        # Create plot
        plt.figure(figsize=(10, 6))
        plt.plot(df["Time (ps)"], df["RMSD (Å)"])
        plt.xlabel("Time (ps)")
        plt.ylabel("RMSD (Å)")
        plt.title(f"RMSD - {selection} - {traj_base}")

        # Save plot to output directory
        plot_path = os.path.join(output_dir, f"{traj_base}_rmsd_{selection_name}_plot.png")
        plt.savefig(plot_path, dpi=300)
        print(f"Plot saved to: {plot_path}")

        # Show the plot
        plt.show()
        
        return df
    except Exception as e:
        print(f"Error running custom RMSD analysis: {e}")
        return None

custom_rmsd_button = widgets.Button(
    description='Run Custom RMSD',
    button_style='info', 
    tooltip='Click to run RMSD with custom selection'
)

custom_rmsd_button.on_click(lambda b: run_custom_rmsd_analysis())
display(custom_rmsd_button)

In [ ]:
def run_custom_rmsf_analysis():
    valid, psf_file, traj_file, output_dir, traj_base = check_input_files()
    if not valid:
        return
    
    selection = rmsf_selection.value
    if not selection.strip():
        print("Error: Please enter a valid atom selection")
        return
    
    try:
        # Load data
        u = mda.Universe(psf_file, traj_file)
        selected_atoms = u.select_atoms(selection)
        
        if len(selected_atoms) == 0:
            print(f"Error: Selection '{selection}' returned no atoms")
            return

        # Run RMSF analysis
        rmsf = RMSF(selected_atoms)
        rmsf.run()

        # Create DataFrame
        atom_identifiers = [f"{atom.resname}{atom.resid}_{atom.name}" for atom in selected_atoms]
        df = pd.DataFrame({
            "Atom": atom_identifiers,
            "Residue": selected_atoms.resids,
            "RMSF (Å)": rmsf.results.rmsf
        })

        # Save data to CSV in output directory
        selection_name = selection.replace(" ", "_").replace("*", "all")
        csv_path = os.path.join(output_dir, f"{traj_base}_rmsf_{selection_name}.csv")
        df.to_csv(csv_path, index=False)
        print(f"CSV data saved to: {csv_path}")

        # Create plot
        plt.figure(figsize=(10, 6))
        plt.plot(df.index, df["RMSF (Å)"])
        plt.xlabel("Atom Index")
        plt.ylabel("RMSF (Å)")
        plt.title(f"RMSF - {selection} - {traj_base}")

        # Save plot to output directory
        plot_path = os.path.join(output_dir, f"{traj_base}_rmsf_{selection_name}_plot.png")
        plt.savefig(plot_path, dpi=300)
        print(f"Plot saved to: {plot_path}")

        # Show the plot
        plt.show()
        
        return df
    except Exception as e:
        print(f"Error running custom RMSF analysis: {e}")
        return None

custom_rmsf_button = widgets.Button(
    description='Run Custom RMSF',
    button_style='info', 
    tooltip='Click to run RMSF with custom selection'
)

custom_rmsf_button.on_click(lambda b: run_custom_rmsf_analysis())
display(custom_rmsf_button)

In [ ]:
def run_custom_rgyr_analysis():
    valid, psf_file, traj_file, output_dir, traj_base = check_input_files()
    if not valid:
        return
    
    selection = rgyr_selection.value
    if not selection.strip():
        print("Error: Please enter a valid atom selection")
        return
    
    try:
        # Load data
        u = mda.Universe(psf_file, traj_file)
        selected_atoms = u.select_atoms(selection)
        
        if len(selected_atoms) == 0:
            print(f"Error: Selection '{selection}' returned no atoms")
            return

        # Calculate RGYR for each frame
        rgyr = []
        times = []

        for ts in u.trajectory:
            rgyr.append(selected_atoms.radius_of_gyration())
            times.append(ts.time)

        # Create DataFrame
        df = pd.DataFrame({
            "Time (ps)": times,
            "Radius of Gyration (Å)": rgyr
        })

        # Save data to CSV in output directory
        selection_name = selection.replace(" ", "_").replace("*", "all")
        csv_path = os.path.join(output_dir, f"{traj_base}_rgyr_{selection_name}.csv")
        df.to_csv(csv_path, index=False)
        print(f"CSV data saved to: {csv_path}")

        # Create plot
        plt.figure(figsize=(10, 6))
        plt.plot(times, rgyr)
        plt.xlabel("Time (ps)")
        plt.ylabel("Radius of Gyration (Å)")
        plt.title(f"Radius of Gyration - {selection} - {traj_base}")

        # Save plot to output directory
        plot_path = os.path.join(output_dir, f"{traj_base}_rgyr_{selection_name}_plot.png")
        plt.savefig(plot_path, dpi=300)
        print(f"Plot saved to: {plot_path}")

        # Show the plot
        plt.show()
        
        return df
    except Exception as e:
        print(f"Error running custom RGYR analysis: {e}")
        return None

custom_rgyr_button = widgets.Button(
    description='Run Custom RGYR',
    button_style='info', 
    tooltip='Click to run RGYR with custom selection'
)

custom_rgyr_button.on_click(lambda b: run_custom_rgyr_analysis())
display(custom_rgyr_button)